User story 4.<br><br>
Om te bepalen welk model we gaan gebruiken moeten we weten wat het verschil is in die modellen en wat het beste past bij het probleem en wat in de beste score resulteerd. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import datetime as date

from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

n_jobs = 16 #set number of jobs equal or -1 the amount of cpu cores you have to speed up this notebook.

Variable prep

In [2]:
usefullcols = ['stm_mon_nr','stm_sap_meld_ddt','stm_status_melding_sap','stm_km_van_mld','stm_km_tot_mld','stm_km_van_gst',
'stm_km_tot_gst','stm_fh_tijd','stm_sap_melddatum','stm_aanngeb_tijd','stm_aanntpl_tijd','stm_arbeid',
'stm_progfh_in_tijd','stm_progfh_in_invoer_tijd','stm_progfh_in_duur','stm_sap_storeindtijd','stm_progfh_gw_tijd',
'stm_reactie_duur','stm_progfh_gw_duur','stm_progfh_gw_teller','stm_afspr_aanvangdd','stm_afspr_aanvangtijd',
'stm_fh_duur','stm_evb','stm_sap_meldtijd','stm_sap_meldtekst_lang','stm_prioriteit','stm_oh_pg_gst',
'stm_sap_meldtekst','stm_techn_gst','stm_contractgeb_gst','stm_tao_indicator','stm_geo_mld','stm_functiepl_mld',
'stm_geo_mld_uit_functiepl','stm_aanngeb_ddt','stm_aanngeb_dd','stm_oorz_code','stm_oorz_groep','stm_oorz_tkst',
'stm_fh_dd','stm_fh_status','stm_geo_gst','stm_functiepl_gst','stm_geo_gst_uit_functiepl','stm_fh_ddt',
'stm_aanntpl_dd','stm_techn_mld','stm_sap_storeinddatum','stm_equipm_nr_mld','stm_equipm_soort_mld',
'stm_equipm_omschr_mld','stm_sap_storeind_ddt','stm_contractgeb_mld','stm_equipm_nr_gst','stm_equipm_soort_gst',
'stm_equipm_omschr_gst','stm_progfh_in_invoer_dat','stm_progfh_in_datum','stm_oorz_tekst_kort','stm_dstrglp_naar',
'stm_tao_indicator_vorige','stm_vl_post','stm_dstrglp_van','stm_pplg_van','stm_tao_soort_mutatie',
'stm_progfh_gw_lwd_tijd','stm_pplg_naar','stm_progfh_gw_lwd_datum']

In [3]:
df = pd.read_csv('sap_storing_data_hu_project.csv',usecols=usefullcols, low_memory=True)
# df = pd.read_csv('sap_storing.csv',usecols=usefullcols, low_memory=True)

C:\Python383\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (11,24,27,49,51,55,56,73,75,78,80,82,86,87,91,95,123,124) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
features = ['stm_prioriteit','stm_oorz_code','stm_oorz_groep','stm_equipm_nr_*','stm_equipm_soort_*','stm_geo_*']
target = 'duur'

In [5]:
now = date.datetime.today()
zeroTimeDiff = now - now

df.drop_duplicates(subset=['stm_mon_nr'], keep='last',inplace = True)

df['stm_aanntpl_ddt'] = pd.to_datetime(df['stm_aanntpl_dd']+' '+df['stm_aanntpl_tijd'], format='%d/%m/%Y %H:%M:%S')
df['stm_fh_ddt'] = pd.to_datetime(df['stm_fh_ddt'], format='%d/%m/%Y %H:%M:%S')

df.dropna(subset=['stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt'],inplace=True)

df['duur'] = df['stm_fh_ddt'] - df['stm_aanntpl_ddt']
df.dropna(subset=['duur'],inplace=True)

# Meldingen waar de oplossing in de toekomst is zijn onzin.
# df[df['stm_sap_storeind_ddt'] >= now][['stm_sap_storeind_ddt','stm_sap_meld_ddt']]
# df.drop(df[df['stm_sap_storeind_ddt'] >= now][['stm_sap_storeind_ddt','stm_sap_meld_ddt']].index,inplace=True)

# Rijen waar het probleem sneller is opgelost dan 5 minuten (en ook de meldingen tijd) zijn onbetrouwbaar.
df.drop(df[df['duur'] <= date.timedelta(minutes=5)].index, inplace = True)

# # Alles wat langer dan een dag duurt is niet urgent genoeg en dus niet nuttig.
df.drop(df[df['duur'] > date.timedelta(hours=6)].index, inplace = True)

In [6]:
display(df[['duur']])
print(df['duur'].shape)

,duur
32099,00:40:00
60683,00:15:00
85621,00:35:00
91875,03:40:19
92852,00:55:00
...,...
898355,00:18:00
898368,01:30:00
898413,00:39:54
898475,00:58:00


(66657,)


Merge similar columns together.

In [7]:
dubbleCol = ['stm_equipm_omschr_*', 'stm_equipm_soort_*', 'stm_equipm_nr_*', 
             'stm_geo_*_uit_functiepl', 'stm_functiepl_*', 'stm_geo_*',
            'stm_km_van_*','stm_km_tot_*',
            'stm_contractgeb_*','stm_techn_*']
original = 'mld'
optional = ['gst'] # Order of least to most important
for colPH in dubbleCol:
    colOg = colPH.replace('*',original)
    
    for option in optional:
        colOp = colPH.replace('*',option)
        df[colPH] = np.where(df[colOp].isna(), df[colOg], df[colOp])
        df.drop(columns=[colOp],inplace=True)
    df.drop(columns=[colOg],inplace=True)

Het vullen van interresante colommen.
Correcte type waardes geven aan de colommen.

In [8]:
df['stm_prioriteit'] = df['stm_prioriteit'].fillna(1) # Laagste prio
df['stm_oorz_code'] = df['stm_oorz_code'].fillna(299) # 'Niet gemeld' code
df['stm_oorz_groep'] = df['stm_oorz_groep'].fillna('x')
df['stm_fh_status'] = df['stm_fh_status'].fillna(1) # Median of values
df['stm_aanngeb_tijd'] = df['stm_aanngeb_tijd'].fillna(df['stm_sap_meldtijd']) # Aannemer beltijd gelijk met melding zetten.
df['stm_progfh_in_invoer_tijd'] = df['stm_progfh_in_invoer_tijd'].fillna(df['stm_aanngeb_tijd']) # Tijd van prognose op beltijd zetten.
df['stm_equipm_nr_*'] = df['stm_equipm_nr_*'].fillna(0)
df['stm_equipm_soort_*'] = df['stm_equipm_soort_*'].fillna('x')
df['stm_contractgeb_*'] = df['stm_contractgeb_*'].fillna(0)
df['stm_progfh_gw_teller'] = df['stm_progfh_gw_teller'].fillna(0)

df['stm_prioriteit'] = df['stm_prioriteit'].astype(int)
df['stm_oorz_code'] = df['stm_oorz_code'].astype(int)
df['stm_fh_status'] = df['stm_fh_status'].astype(int)
df['stm_fh_duur'] = df['stm_fh_duur'].astype(int)
df['stm_progfh_gw_teller'] = df['stm_progfh_gw_teller'].astype(int)
df['stm_equipm_nr_*'] = df['stm_equipm_nr_*'].astype(int)
df['stm_contractgeb_*'] = df['stm_contractgeb_*'].astype(int)

df['duur'] = df['duur'].apply(lambda x: x.seconds / 60)
df['duur'] = df['duur'].astype(int)

Baseline model 

In [9]:
amntInTime = df[df['duur'] < date.timedelta(hours=2,minutes=17).seconds / 60].count()[0]
total = df.shape[0]
baseline = date.timedelta(hours=2,minutes=17).seconds / 60
pred = [int(baseline)]*df[['duur']].shape[0]

print(f'{amntInTime / total * 100}% van de tijd heeft het baseline model het goed.')
print('Root mean squared error:',np.sqrt(mean_squared_error(df['duur'],pred)))

86.47103830055359% van de tijd heeft het baseline model het goed.
Root mean squared error: 95.1771464162169


In [10]:
features

['stm_prioriteit',
 'stm_oorz_code',
 'stm_oorz_groep',
 'stm_equipm_nr_*',
 'stm_equipm_soort_*',
 'stm_geo_*']

In [11]:
le_ozgr = LabelEncoder()
le_ozgr.fit(df['stm_oorz_groep'].unique())
df['stm_oorz_groep'] = le_ozgr.transform(df['stm_oorz_groep'])
# df['stm_oorz_groep'].unique()

In [12]:
le_equipsrt = LabelEncoder()
le_equipsrt.fit(df['stm_equipm_soort_*'].unique())
df['stm_equipm_soort_*'] = le_equipsrt.transform(df['stm_equipm_soort_*'])
# df['stm_equipm_soort_*'].unique()

In [13]:
dOC = pd.get_dummies(df['stm_oorz_code'],prefix='oorz_code')
dOG = pd.get_dummies(df['stm_oorz_groep'],prefix='oorz_groep')
eqs = pd.get_dummies(df['stm_equipm_soort_*'])
geo = pd.get_dummies(df['stm_geo_*'],prefix='geo')

dummies = pd.concat([df['stm_prioriteit'],geo,dOC, dOG, eqs],axis=1)

display(dummies)
dummies.shape

,stm_prioriteit,geo_1.0,geo_2.0,geo_3.0,geo_4.0,geo_5.0,geo_6.0,geo_7.0,geo_8.0,geo_9.0,...,281,282,283,284,285,286,287,288,289,290
32099,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
60683,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
85621,9,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
91875,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
92852,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898355,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
898368,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
898413,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
898475,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(66657, 975)

In [14]:
# display(df[[target]])
# display(df[[target]].describe())
# print(df[[target]].shape)

In [15]:
X = dummies
# X = df[features]
Y = df[target]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=21)

In [16]:
print(y_train,y_train.shape)

166704     40
469546     52
752187     60
246706     19
596457     32
         ... 
817163    210
492952     17
463751     33
810326     25
235216     10
Name: duur, Length: 59991, dtype: int32 (59991,)


In [17]:
linearModel = linear_model.LinearRegression().fit(x_train,y_train)
linYPrediction = linearModel.predict(x_test)
print(f'Score: {linearModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,linYPrediction))}')

Score: -3.1650767882092027e+18
Mean squared error: 118768855200.01678


In [18]:
display(x_test[:5])
display(y_test[:5])
linYPrediction[:5]

,stm_prioriteit,geo_1.0,geo_2.0,geo_3.0,geo_4.0,geo_5.0,geo_6.0,geo_7.0,geo_8.0,geo_9.0,...,281,282,283,284,285,286,287,288,289,290
436311,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
241351,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
666482,5,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
524448,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372190,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


436311    103
241351    126
666482     60
524448      8
372190    294
Name: duur, dtype: int32

array([63.58203125, 87.1171875 , 88.8671875 , 36.9609375 , 45.88671875])

In [19]:
ridgeModel = linear_model.Ridge(alpha=1).fit(x_train,y_train)
ridgeYPrediction = ridgeModel.predict(x_test)
print(f'Score: {ridgeModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,ridgeYPrediction))}')
print(ridgeModel.get_params())
print(ridgeModel.get_params(deep=True))

Score: 0.12854153794855272
Mean squared error: 62.320910627443546
{'alpha': 1, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}
{'alpha': 1, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}


In [20]:
# knearestModel = KNeighborsRegressor(n_neighbors=2,n_jobs=n_jobs).fit(x_train,y_train)
# knearestYPrediction = knearestModel.predict(x_test)
# print(f'Score: {knearestModel.score(x_test,y_test)}')
# print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,knearestYPrediction))}')

In [21]:
# max_neighbours = 10
# scores_train = np.zeros(max_neighbours)
# scores_test = np.zeros(max_neighbours)
# variance = np.zeros(max_neighbours)

# neighbours = range(1, max_neighbours + 1)
# for n in neighbours:
#     print(f"Starting with run:{n}")
#     model = KNeighborsRegressor(n_neighbors=n,n_jobs=n_jobs,leaf_size=10)
#     model.fit(x_train, y_train)
#     score_train = model.score(x_train, y_train)
#     scores_train[n-1] = score_train

#     score_test = model.score(x_test, y_test)
#     scores_test[n-1] = score_test

#     variance[n-1] = score_train - score_test
#     #     print("Score bij diepte {}: {:.2f}%.".format(depth, score))

# fig, ax = plt.subplots(1, dpi=100)

# line1, = ax.plot(neighbours, scores_train, label='Train')
# line2, = ax.plot(neighbours, scores_test, label='Test')
# line3, = ax.plot(neighbours, variance, label='Variance')

# ax.legend()
# ax.set_xlabel("diepte (aantal lagen)")
# ax.set_ylabel("Bias-Variance")

# plt.show()

## Non-linear regression trees

In [22]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [23]:
dtree1 = DecisionTreeRegressor(max_depth=2)
dtree2 = DecisionTreeRegressor(max_depth=37)
dtree1.fit(x_train, y_train)
dtree2.fit(x_train, y_train)

# Code Lines 5 to 6: Predict on training data
tr1 = dtree1.predict(x_train)
tr2 = dtree2.predict(x_train) 

#Code Lines 7 to 8: Predict on testing data
y1 = dtree1.predict(x_test)
y2 = dtree2.predict(x_test) 

In [24]:
print(np.sqrt(mean_squared_error(y_train,tr1))) 
print(r2_score(y_train, tr1))

print(np.sqrt(mean_squared_error(y_test,y1))) 
print(r2_score(y_test, y1))

65.37651944532037
0.03875749516914739
65.54789427157387
0.03595657000510666


In [25]:
print(np.sqrt(mean_squared_error(y_train,tr2))) 
print(r2_score(y_train, tr2))

print(np.sqrt(mean_squared_error(y_test,y2))) 
print(r2_score(y_test, y2))

53.97559847364261
0.34478428067593414
66.1155301751796
0.019187301678678104


In [26]:
# max_depth = 20
# scores_train = np.zeros(max_depth)
# scores_test = np.zeros(max_depth)
# variance = np.zeros(max_depth)

# depths = range(1, max_depth + 1)
# for depth in depths:
#     model = DecisionTreeRegressor(max_depth=depth, min_samples_split=20)
#     model.fit(x_train, y_train)
    
#     score_train = model.score(x_train, y_train)
#     scores_train[depth-1] = score_train

#     score_test = model.score(x_test, y_test)
#     scores_test[depth-1] = score_test

#     variance[depth-1] = score_train - score_test
#     #     print("Score bij diepte {}: {:.2f}%.".format(depth, score))

# fig, ax = plt.subplots(1, dpi=100)

# line1, = ax.plot(depths, scores_train, label='Train')
# line2, = ax.plot(depths, scores_test, label='Test')
# line3, = ax.plot(depths, variance, label='Variance')

# ax.legend()
# ax.set_xlabel("diepte (aantal lagen)")
# ax.set_ylabel("Bias-Variance")

# plt.show()

In [27]:
dtree_final = DecisionTreeRegressor(max_depth=8)
dtree_final.fit(x_train, y_train)

y_final = dtree_final.predict(x_test)

print(r2_score(y_test, y_final))
print(np.sqrt(mean_squared_error(y_test,y_final))) 

0.0930168093585666
63.57846875710983


In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [29]:
model_rf = RandomForestRegressor(max_features=None, n_estimators=50, max_leaf_nodes=48, oob_score=True, random_state=100,n_jobs=n_jobs)

In [30]:
model_rf.fit(x_train, y_train) 
pred_train_rf= model_rf.predict(x_train)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print('Train R^2 score:',r2_score(y_train, pred_train_rf))
pred_test_rf= model_rf.predict(x_test)
print('Test RMSE:',np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print('Test R^2 score:',r2_score(y_test, pred_test_rf))

Train RMSE: 62.25820195528458
Train R^2 score: 0.1282689344935346
Test RMSE: 62.29861607909364
Test R^2 score: 0.12916493376924598


In [31]:
model = RandomForestRegressor(
    max_features=None, n_estimators=50, max_leaf_nodes=48, oob_score=True, random_state=100,n_jobs=n_jobs
                             ).fit(x_train, y_train)

In [32]:
def random_forest_regressor_predict_proba(X_train, y_train, X_test, m):
    """Trains DecisionTreeRegressor model and predicts probabilities of each y.

    Args:
        X_train: Training features.
        y_train: Training labels.
        X_test: New data to predict on.
        **kwargs: Other arguments passed to DecisionTreeRegressor.

    Returns:
        DataFrame with columns for record_id (row of X_test), y 
        (predicted value), and prob (of that y value).
        The sum of prob equals 1 for each record_id.
    """
    # Get y values corresponding to each node.
    node_ys = pd.DataFrame({'node_id':m.apply(X_train)[:,0], 'duur': y_train})
    # Calculate probability as 1 / number of y values per node.
    node_ys['prob'] = 1 / node_ys.groupby(node_ys.node_id).transform('count')
    # Aggregate per node-y, in case of multiple training records with the same y.
    node_ys_dedup = node_ys.groupby(['node_id', 'duur']).prob.sum().to_frame()\
        .reset_index()
    
    dept = m.decision_path(X_test)[1][1]
    leaf = pd.DataFrame(m.decision_path(X_test)[0].toarray()[:,:dept]).apply(
        lambda x:x.to_numpy().nonzero()[0].max(), axis=1).to_frame(
            name='node_id')
    leaf['record_id'] = leaf.index
    # Merge with y values and drop node_id.
    return leaf.merge(node_ys_dedup, on='node_id').drop(
        'node_id', axis=1).sort_values(['record_id', 'duur'])

In [35]:
res = random_forest_regressor_predict_proba(x_train, y_train, x_test[:1],model)

In [36]:
res[res['record_id'] == 0].sort_values('prob',ascending=False)

,record_id,duur,prob
25,0,30,0.022603
10,0,15,0.020653
5,0,10,0.019810
20,0,25,0.017492
15,0,20,0.017176
...,...,...,...
341,0,351,0.000053
297,0,302,0.000053
343,0,353,0.000053
326,0,334,0.000053


In [ ]:
# filename = 'finalized_model_lin.sav'
# pickle.dump(dtree_final, open(filename, 'wb'))
# filename = 'finalized_model_dectreereg.sav'
# pickle.dump(dtree_final, open(filename, 'wb'))
# filename = 'finalized_model_knearest.sav'
# pickle.dump(knearestModel, open(filename, 'wb'))
# df.to_csv('sap_storing_data_hu_project_norm.csv', columns=features)

filename = 'rand_for.sav'
pickle.dump(model_rf, open(filename, 'wb'))

In [ ]:
baseline = date.timedelta(hours=2,minutes=17)
baseline = baseline.seconds / 60
pred = [int(baseline)]*df[[target]].shape[0]
r2_score(df[[target]],pred)